# Fair Weather pedalers
** Weather's effect on bike share ridership on the Hubway system in Boston and its environs. **  
Kevin Burek &lt;<kburek@fas.harvard.edu>>, Joshua Mclellan &lt;<jvl.mclellan@gmail.com>>  
Harvard AM 207 Spring 2016 Final Project  

## Abstract
The Hubway bicycle share system serves riders in the urban core and near suburbs of Boston.  We'd like to discover something about the riders who use the system.  In 2013, the service providers released a dataset which we will analyze with respect to historical weather data, in order to explore the validity of different modeling hypotheses.  In brief, we believe that ridership may be composed of different constituencies, commuters and joyriders, which are affected differently by varying weather conditions.

## Tools
For analysis, we are using SciPy, NumPy, Pandas, PyMC, and Matplotlib.  
Some other core python libaries are coming along for the ride, too.

In [18]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")

import time
import timeit

import dateutil.parser as dtp

import scipy.stats as stats
import pandas as pd
import pymc as pm

import re
import numpy as np

import string

## Data
The good folks at [Hubway](http://www.thehubway.com/) published the corpus of ridership logs from the system's inception in July 2011 through the end of the cycling season in 2013<sup>[[1]][@hubwaydatachallenge_zip]</sup>. For weather data, we turn to NOAA, whose [National Centers for Environmental Information](https://www.ncdc.noaa.gov/) provides the service of publishing [Climate Data Online](https://www.ncdc.noaa.gov/cdo-web/). We requested and received a data set including daily observed weather conditions for the relevant time period, for the Boston metropolitan area<sup>[[2]][@ncdc_boston]</sup>.


[@hubwaydatachallenge_zip]: http://files.hubwaydatachallenge.org/hubway_2011_07_through_2013_11.zip "Hubway Data Challenge. "hubway_2011_07_through_2013_11". Zip Archive. Retrieved 16 March 2016."

[@ncdc_boston]: http://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/locations/CITY:US250002/detail "NOAA National Centers for Environmental Information. "Daily Summaries Location Details: Boston, MA""

In [19]:
# Set a flag for whether to load the whole data set, or just a portion.
load_all = False
nb_root = ""
# Read ridership data
rides_path = "data/hubway_2011_07_through_2013_11/%shubway_trips.csv"
raw_rides = pd.read_csv(nb_root + rides_path % "") if load_all else pd.read_csv(nb_root + rides_path % "fewer_")
# Read weather data
weather_na = ("unknown", "9999", "-9999")
raw_weather = pd.read_csv(nb_root + "data/ncdc-2013.csv", na_values=weather_na)

raw_weather.head()

,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,DATETIME,PRCP,PRCP Measurement Flag,PRCP Quality Flag,...,TMIN,TMIN Measurement Flag,TMIN Quality Flag,TMIN Source Flag,TMIN Time of Observation,TOBS,TOBS Measurement Flag,TOBS Quality Flag,TOBS Source Flag,TOBS Time of Observation
0,GHCND:USW00054704,NORWOOD MEMORIAL AIRPORT MA US,15.2,42.19083,-71.17361,20100101,2010-01-01T12:00,0,T,,...,-38,,,W,NaN,NaN,,,,NaN
1,GHCND:USW00054704,NORWOOD MEMORIAL AIRPORT MA US,15.2,42.19083,-71.17361,20100102,2010-01-02T12:00,23,,,...,-60,,,W,NaN,NaN,,,,NaN
2,GHCND:USW00054704,NORWOOD MEMORIAL AIRPORT MA US,15.2,42.19083,-71.17361,20100103,2010-01-03T12:00,5,,,...,-93,,,W,NaN,NaN,,,,NaN
3,GHCND:USW00054704,NORWOOD MEMORIAL AIRPORT MA US,15.2,42.19083,-71.17361,20100104,2010-01-04T12:00,0,,,...,-88,,,W,NaN,NaN,,,,NaN
4,GHCND:USW00054704,NORWOOD MEMORIAL AIRPORT MA US,15.2,42.19083,-71.17361,20100105,2010-01-05T12:00,0,,,...,-93,,,W,NaN,NaN,,,,NaN


In [20]:
weather = pd.DataFrame();
weather['STATION'] = raw_weather['STATION']
weather['DATE'] = raw_weather['DATE']
weather.set_index(['STATION', 'DATE'], drop=True, inplace=True)
weather['YEAR'] = [d[0:4] for d in raw_weather['DATE'].astype(np.str_)]
weather['MONTH'] = [d[4:6] for d in raw_weather['DATE'].astype(np.str_)]
weather['DAY'] = [d[6:] for d in raw_weather['DATE'].astype(np.str_)]
weather.head()


YEAR MONTH DAY
STATION           DATE                    
GHCND:USW00054704 20100101  2010    01  01
                  20100102  2010    01  02
                  20100103  2010    01  03
                  20100104  2010    01  04
                  20100105  2010    01  05

## Cleaning
Here we clean the data for analysis.  

### Augmenting ridership data
The ridership data has some properties that aren't well expressed in its current form. We calculate and add columns for the following extracted properties:  
* Start & End day of week (0/Monday - 6/Sunday) 
* Start & End time of day (00:00 - 23:59)
* ... Duration, rider age, ...

In [21]:
do_rebuild = False
ride_pickle_name = 'rides_pickle'
try:
    if do_rebuild: raise 'abort.';
    rides = pd.read_pickle(ride_pickle_name)
except:
    import dateutil.parser as dtp
    rides = raw_rides.copy()
    totalseconds = lambda t: (t.hour*60 + t.minute)*60 + t.second
    for endpoint in ('start', 'end'):
        %time datetimes = [dtp.parse(d) for d in rides[endpoint + '_date']]
        rides.loc[:,endpoint + '_datetime'] = pd.Series(datetimes, index = rides.index)
        rides.loc[:,endpoint + '_weekday'] = pd.Series([d.weekday() for d in datetimes], index = rides.index)
        rides.loc[:,endpoint + '_seconds'] = pd.Series([totalseconds(d.time()) for d in datetimes], index = rides.index)
    
    rides.to_pickle(ride_pickle_name)




Wall time: 12.8 s
Wall time: 15.1 s


### Augmenting weather data
The raw weather data needs measurement flag columns renamed to associate them with the measurement dimensions they describe.  For now, we use pre-processed data.


# Formulating the likelihood
To generate a posterior predictive, we will eventually need
$$ P(\Theta | D) = \frac{P(D|\Theta)P(\Theta)}{P(D)} $$
The likelihood is a product over all of our ridership data $R$ and spread across heirarchical model components $L$ for each rental location:
$$ \mathscr{L} = P(D | \Theta) = \prod_R { \sum_L { p\left(t_d, w_d, \omega, s\ |\ \mu_t, \sigma_t, \lambda_d, c\right) } } $$

With data dimensions:  
$t_d$: Time of departure $\in$ $[$0/Midnight $\dots$ 43200/Noon $\dots$ 86400/Midnight$)$  
$w_d$: Weekday of departure $\in$ $\{$0/Monday $\dots$ 6/Sunday$\}$  
$\omega$: Weather quality $\propto {f}($temperature, sunlight, $\dots)$  
$s$: Subscription status $\in$ $\{0,1\}$

With model parameters:  
$\mu_t$: Mean time of departure $\in$ $[$0/Midnight $\dots$ 43200/Noon $\dots$ 86400/Midnight$)$  
$\sigma_t$: Deviation in time of departure $\in$ $[0,\infty)$  
$\lambda_w$: Ratio of weekday departures $\in$ $[0,1)$ such that $\left(\sum_{i=0}^{6}{\lambda_{w_i}}\right) = 1$  
$c$: Rider Commitment $\in$ $\{$0/Joyrider, 1/Commuter$\}$  


The likelihood is a beast.  First, we can write the location-likelihood as a linear function of the fair-weather likelihood:  
$$\mathscr{L}_L = \omega(\mathscr{L}_f) + b + \varepsilon$$
The fair-weather likelihood is mixture of gaussians:  
$$\begin{align}  
\mathscr{L}_f
= & c_0\cdot\left(\sum_w{\lambda_w\left(\mathcal{N}\left(t_d\mid\mu_{t_{c_0}L}, \sigma_{t_{c_0}}\right)\right)}\right) 
+ c_1\cdot\left(\sum_w{\lambda_w\left(\mathcal{N}\left(t_d\mid\mu_{t_{c_1}L}, \sigma_{t_{c_\overline{0}}}\right)\right)}\right)
+ c_2\cdot\left(\sum_w{\lambda_w\left(\mathcal{N}\left(t_d\mid\mu_{t_{c_2}L}, \sigma_{t_{c_\overline{0}}}\right)\right)}\right)
\end{align}$$

[expanded_exponentials]: mathTeX:saved/for/later "(1-c)\cdot\left(\sum_w{\lambda_w\left(exp\left[\frac{-(t_d-\mu_{t_\overline{c}L})^2}{{\sigma_{t_{\overline{c}}}}^2}\right]\right)}\right) + \frac{c}{2}\cdot\left(\sum_w{\lambda_w\left(exp\left[\frac{-(t_d-\mu_{t_{c1}L})^2}{{\sigma_{t_c}}^2}\right] + exp\left[\frac{-(t_d-\mu_{t_{c2}L})^2}{{\sigma_{t_c}}^2}\right]\right)}\right) \\ "

## Labeling commitment on weekday rides

We will use expectation maximization to attempt recovery of the hidden commitment label $c$.

The E-step will find the expectation values of the mixure ratios, $c_i$.


The M-step will maximize the likelihood with respect to the parameters of the mixture components, $\mu_{t_{c_i}}$ and $\sigma_{t_{c_i}}$



In [22]:
# Find weights for each c_i given the data and current mixture parameters.
def e_step(data, mu, sigma, lam):
    """
    Find weights for each c_i given the data and current mixture parameters
    :data: pandas dataframe with 'start_seconds'
    :mu: array of 3 of mixture time means.
    :sigma: array of 2 of mixture time deviations.
    :return: weights [c_0, c_1, c_2] for each item in data.
    """
    td = np.array(data['start_seconds']) # Time of departure
    c0 = stats.norm.pdf(td, mu[0], sigma[0])
    c1 = stats.norm.pdf(td, mu[1], sigma[1])
    c2 = stats.norm.pdf(td, mu[2], sigma[1])
    cs = np.array([c0, c1, c2])/np.sum(np.array([c0,c1,c2]), axis=0)  # Normalized array of c weights.
    return cs


c_weights = e_step(rides, [14*60*60,8*60*60,17*60*60], [3*60*60, 1*60*60], None)

# Update mixture parameters given the data and the current mixture weights.
def m_step(data, cs):
    """
    Update mixture parameters given the data and the current mixture weights.
    :param data: pandas dataframe with 'start_seconds'
    :param cs: array of 3 mixture weights.
    :return: hash {'mu', 'sigma'}
    """
    #TODO: Update mixture parameters using MLE.
    td = np.array(data['start_seconds'])
    sig = np.array([3*60*60, 1*60*60])

    def log_like(w, m, s):
        return np.sum(w*(stats.norm.logpdf(td, m, s) - np.log(w)))

    mus = np.arange(0, 24*60*60, 30)
    ll0 = [log_like(cs[0], mu, sig[0]) for mu in mus]
    mu_0 = mus[np.argmax(ll0)]
    ll1 = [log_like(cs[1], mu, sig[1]) for mu in mus]
    mu_1 = mus[np.argmax(ll1)]
    ll2 = [log_like(cs[2], mu, sig[1]) for mu in mus]
    mu_2 = mus[np.argmax(ll2)]
    return {'mu': np.array([mu_0, mu_1, mu_2]), 'sigma': sig}


In [ ]:
# Run E-M iteration until the weights don't change, or we hit 100 iterations.

itr = 0
old_c_weights = np.zeros(c_weights.shape)
while np.sum(np.abs(c_weights - old_c_weights)) > .0001:
    itr += 1
    old_c_weights = c_weights
    parms = m_step(rides, c_weights)
    print 'Model parameters: %s' % str(parms)
    c_weights = e_step(rides, parms['mu'], parms['sigma'], None)
    print 'Weights: %s' % str(c_weights)
    if itr > 100:
        break

In [16]:
e_step(rides, [14*60*60,8*60*60,17*60*60], [3*60*60, 1*60*60], None)

NameError: global name 'stats' is not defined

(array([  74,  179,  329,  805, 1235, 1081, 1316, 1768, 2576, 2258, 1942,
       2641, 2038, 2105, 2028, 1558, 1374, 1728, 1382, 1478, 1381, 1262,
        946,  900, 1408,  833,  953,  881, 1318,  793,  580,  819, 1016,
        521,  718,  758,  564,  777,  527,  405,  293,  457,  282,  189,
        286,  181,  150,  202,  114,  125,   41,   23,   54,    5,   27,
         14,   13,    0,    0,    0,    0,    5], dtype=int64), array([ 17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,
        28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,
        39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,
        50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,
        61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,
        72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.]))


In [79]:
rides = rides.dropna()
ages = [d.year for d in rides['start_datetime']] - rides['birth_date']

nbins = np.max(ages) - np.min(ages)
print np.histogram(ages, bins=nbins)
plt.hist(ages, bins=nbins)
plt.show()


79.0


35.6460395048


(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan]))

In [81]:
28.78*1.18

33.9604